In [ ]:
#Imports

#Plot
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

#Coco dataset
import cocoex, cocopp
import bbobbenchmarks as bn

#Solver scipy
import scipy
from scipy.optimize import fmin

#Common libraries
import numpy as np
import random
import sys

#My notebooks
import nbimporter
from metamodel import *

In [ ]:
#-------------------------------------------------

In [ ]:
def select_population(individuals, size, mode="Gagne"):
    
    if mode == "Gagne":
        if len(individuals) < size:

            choices = np.random.choice(len(individuals), size)
            curr_pop = np.array([individuals[i] for i in choices])
            return curr_pop
        else:
            #50% top:
            top50 = sorted(range(len(individuals)), key=lambda i: individuals[i], reverse=True)[-size//2:]

            #50% random:
            random50 = [random.randint(0, len(individuals)-1) for i in range(size//2)]

            all_indexes = sorted(top50 + random50)
            curr_pop = [(individuals[ind][0], individuals[ind][1], individuals[ind][2]) for ind in all_indexes]

            return np.array(curr_pop)
    
    elif mode == "random_pop":
        choices = np.random.choice(len(individuals), size)
        curr_pop = np.array([individuals[i] for i in choices])
        return curr_pop
        
        

def random_sample(nb, dimension, domain):
    points = (domain[1] - domain[0]) * np.random.random_sample((nb,dimension)) + domain[0]
    return points

In [ ]:
#Define problem
problem = bn.F1(1)

#Define Optimizer
fmin = scipy.optimize.fmin

In [ ]:
#try:
#    from keras.models import load_model
#    model = load_model('metamodel.h5')
#    model = Model(model)
#except:
model = Model()

#Hyper-Params:
dim_problem = 2
size_hist, size_sample = 5, 10
hist, curr_hist = [], np.array([(0,0,0)]*size_hist)

interval = [-5, 5]

In [ ]:
print(problem)

best_f = 9999.0 # initialize the "best found" - both the function value and the x values
best_x = [None]*dim_problem

steps = []

for i in range(100):
    
    dataset = []

    for j in range(100000):
        
        #sys.stdout.write('\r')
        #sys.stdout.write("[%-100s] %d%%" % ('='*j, j))
        #sys.stdout.flush()

        #1) Select n random points to be evaluated by meta-model
        points = [(random.uniform(interval[0], interval[1]), random.uniform(interval[0], interval[1]))]*5
        
        #2) Evaluate points in meta-model
        evaluations = np.array([model.predict(point, curr_hist)[0] for point in points])

        #3) Get the best point (highest evaluation) (x_t+1)
        best_index = np.argmax(evaluations)
        best_point = points[best_index] 
        #best_index = np.random.randint(0, len(points))
        #best_point = points[best_index]

        #4) Evaluate this point in black-box function
        f_eval = problem(best_point)

        #Save train data:
        #dataset.append(np.array([np.array(best_point), np.array(curr_hist), np.array(f_eval)]))

        #5) Add this point in your history
        hist.append((best_point[0], best_point[1], f_eval))

        #6) Select next current history
        curr_hist = select_population(hist, size_hist)

        if f_eval < best_f: # see if it's an improvement -- in multiobjective, this is the Pareto sort
            best_f = f_eval
            best_x = best_point
            
        #Best point:
        #best = min(hist, key=lambda x: x[2])
        #best_f = round(best[2], 2)

        #Check solution:
        #print(best_f)
        if round(best_f, 2) == problem.fopt:
            steps.append(j)
            break
            
    
        #x0 = (0,0)
        #print(x0)
        #minimum = fmin(problem, x0)
        #print(minimum)
        #print("fopt: ", problem(minimum))
        #print("_")

    #print(best_f)
    #print(problem.fopt)

    #dataset = np.array(dataset[1:])
    #np.save('dataset.npy', dataset)
    
    #Train Model:
    #d = np.load('dataset.npy')
    #model.fit(d)

print(len(steps))
steps = np.array(steps)
print("Mean: ", np.mean(steps), " - Std: ", np.std(steps))

In [ ]:
#iterator = tqdm(range(10000000))
iterator = range(10000000)

for i in iterator:
    pass

In [72]:
class Random_Search_Optimizer:
    
    '''Random Search Optimizer
       
       @init:
       max_steps = max number of iterations
       verbose = True shows progress bar
       
       @optimize:
           *input:
               f = function to be optimized
               warning: 
                   - f.opt = optimal value
                   - f.domain = interval to search, e.g. [-5, 5]
                   - f.dimension = dimension of the function
           *output:
               (best_x, number of steps) if optimal point was found else (None, -1)
    '''
    
    def __init__(self, max_steps=100000, verbose=False):
        self.max_steps = max_steps
        self.verbose = verbose
    
    def optimize(self, f):
        
        #Init best values:
        best_x = [None] * f.dimension 
        best_y = 99999.0

        #Define Iterator:
        if self.verbose:
            iterator = tqdm(range(self.max_steps))
        else:
            iterator = range(self.max_steps)
        
        #Iteration:
        for step in iterator:

            #Choose random x and evaluate it:
            new_x = (f.domain[1] - f.domain[0]) * np.random.random_sample((1, f.dimension)) + f.domain[0]
            new_y = f(new_x)

            #Check if it is an improvement:
            if new_y < best_y: 
                best_y = new_y
                best_x = new_x

                #Check if it is a solution:
                if round(best_y[0], 2) == f.fopt:
                    
                    return (best_x, step)

        return (None, -1)

class Evaluator:
    
    def __init__(self, nb_trials=100, verbose=1):
        self.nb_trials = nb_trials
        self.verbose = verbose

    def evaluate(self, f, optimizer):
        
        sum_x = 0
        sum_x2 = 0
        
        #Define Iterator:
        if self.verbose:
            iterator = tqdm(range(self.nb_trials))
        else:
            iterator = range(self.nb_trials)
        
        #Iteration:
        for trial in iterator:
            bestx, step = optimizer.optimize(f)
            
            sum_x += step
            sum_x2 += step*step

        mean = sum_x / self.nb_trials
        stdev = np.sqrt((sum_x2 / self.nb_trials) - (mean * mean)) 
        
        return {"mean": mean, "stdev": stdev}

In [73]:
import numpy as np
from tqdm import tqdm

import bbobbenchmarks as bn

#Define problem
f = bn.F1(1)
f.domain = [-5, 5]
f.dimension = 2


random_search_opt = Random_Search_Optimizer()
evaluator = Evaluator()

evaluator.evaluate(f, random_search_opt)